<a href="https://colab.research.google.com/github/arka57/Fine-Tuning-BERT/blob/main/Text_Classification_BERT_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing the libraries

In [1]:
!pip install -q transformers
!pip install -q peft
!pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00


In [ ]:
#loading the dataset-->IMDB dataset with two categories from Hugging Face. This dataset is designed for binary sentiment classification

In [2]:
from datasets import load_dataset

dataset=load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
#Pre-processing the text

In [3]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
  return tokenizer(examples["text"],padding="max_length",truncation=True)


tokenize_datasets=dataset.map(tokenize_function,batched=True)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
#Creating a smaller dataset

In [5]:
small_train_dataset = tokenize_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenize_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
#Adding PEFT (LoRA) to the model for efficient fine tuning

#Specifying the task type
#r=Specifying rank/dimension of the 2 matrices
#lora_alpha=scaling factor attributing weightage to lora weight and actual model weights

In [6]:
from peft import LoraConfig,TaskType

lora_config=LoraConfig(task_type=TaskType.SEQ_CLS,r=1,lora_alpha=1,lora_dropout=0.1)

In [ ]:
#Base Model downloaded

In [7]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased',
    num_labels=2
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from peft import get_peft_model
model=get_peft_model(model,lora_config)

In [ ]:
#Specifying evaluation function

In [9]:
import numpy as np
import evaluate

metric=evaluate.load("accuracy")

In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",
                                 num_train_epochs=25,)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.690303,0.532000
2,No log,0.681173,0.599000
3,No log,0.679127,0.546000
4,0.694800,0.655423,0.647000
5,0.694800,0.631244,0.679000
6,0.694800,0.590356,0.706000
7,0.694800,0.547268,0.734000
8,0.608100,0.518282,0.736000
9,0.608100,0.487573,0.764000
10,0.608100,0.457824,0.789000


TrainOutput(global_step=3125, training_loss=0.4720153051757813, metrics={'train_runtime': 2735.959, 'train_samples_per_second': 9.138, 'train_steps_per_second': 1.142, 'total_flos': 6580725657600000.0, 'train_loss': 0.4720153051757813, 'epoch': 25.0})

In [ ]:
#Evaluating the trained model

In [14]:
trainer.evaluate()

{'eval_loss': 0.38556841015815735,
 'eval_accuracy': 0.839,
 'eval_runtime': 36.2515,
 'eval_samples_per_second': 27.585,
 'eval_steps_per_second': 3.448,
 'epoch': 25.0}

In [ ]:
#testing on the data

In [41]:
type(small_train_dataset[0])

dict

In [50]:
import torch

In [68]:
text="There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes.Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have clairvoyance. People like to compare, to judge an evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all..."
encoding = tokenizer(text, return_tensors="pt")
encoding

{'input_ids': tensor([[  101,  1247,  1110,  1185,  6796,  1120,  1155,  1206,  3144,  2852,
          1105, 26890,  1197,  1133,  1103,  1864,  1115,  1241,  1132,  2021,
          1326,  1164,  5973,  6969,   119, 26890,  1197,  2736, 19501,  1183,
           117,  3144,  2852,  2736,  5263,   119, 26890,  1197, 15836,  1132,
          2385,  3014,   119,  3144,  2852,   112,   188,  4928,  1132,  1677,
          1167,  8277,   119,   119,   119,  3144,  2852,  2736,  1167,  1176,
          3460, 15463, 20629,   117,  1191,  1195,  1138,  1106,  3205, 12672,
           119,   119,   119,  1109,  1514,  1959,  1110,  4780,  1105,  6994,
          1186,   117,  1133,  1138,   172, 20293, 12716,  3923,   119,  2563,
          1176,  1106, 14133,   117,  1106,  3942,  1126, 17459,   119,  1731,
          1164,  1198,  8965,   136, 16819,  1645,  1315,   117,  1234,  2269,
          3144,  2852,  2736,  1237,  1133,   117,  1113,  1103,  1168,  1289,
           117,  8995,  1152,  9353,  

In [69]:
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
encoding

{'input_ids': tensor([[  101,  1247,  1110,  1185,  6796,  1120,  1155,  1206,  3144,  2852,
           1105, 26890,  1197,  1133,  1103,  1864,  1115,  1241,  1132,  2021,
           1326,  1164,  5973,  6969,   119, 26890,  1197,  2736, 19501,  1183,
            117,  3144,  2852,  2736,  5263,   119, 26890,  1197, 15836,  1132,
           2385,  3014,   119,  3144,  2852,   112,   188,  4928,  1132,  1677,
           1167,  8277,   119,   119,   119,  3144,  2852,  2736,  1167,  1176,
           3460, 15463, 20629,   117,  1191,  1195,  1138,  1106,  3205, 12672,
            119,   119,   119,  1109,  1514,  1959,  1110,  4780,  1105,  6994,
           1186,   117,  1133,  1138,   172, 20293, 12716,  3923,   119,  2563,
           1176,  1106, 14133,   117,  1106,  3942,  1126, 17459,   119,  1731,
           1164,  1198,  8965,   136, 16819,  1645,  1315,   117,  1234,  2269,
           3144,  2852,  2736,  1237,  1133,   117,  1113,  1103,  1168,  1289,
            117,  8995,  11

In [70]:

outputs = trainer.model(**encoding)

In [71]:
logits = outputs.logits
logits.shape
logits

tensor([[-0.7646,  0.2816]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [72]:
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())

In [73]:
probs

tensor([0.3177, 0.5699], grad_fn=<SigmoidBackward0>)

In [77]:
def sentiment(probs):
  if probs[0]>=0.5:
    print("+ve")
  else:
    print("-ve")

In [78]:
sentiment(probs)

-ve
